<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Let's-try-to-extract-the-list-of-bus-destinations" data-toc-modified-id="Let's-try-to-extract-the-list-of-bus-destinations-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Let's try to extract the list of bus destinations</a></span></li><li><span><a href="#Now,-we-should-find-GPS-coordinates" data-toc-modified-id="Now,-we-should-find-GPS-coordinates-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Now, we should find GPS coordinates</a></span></li><li><span><a href="#Let's-add-wikipedia-links-wia-wikipedia-API" data-toc-modified-id="Let's-add-wikipedia-links-wia-wikipedia-API-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Let's add wikipedia links wia wikipedia API</a></span></li><li><span><a href="#Creating-a-map" data-toc-modified-id="Creating-a-map-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Creating a map</a></span></li></ul></div>

# Beogradska autobuska stancija

### Let's try to extract the list of bus destinations

In [ ]:
import pandas as pd
!pip install folium
import folium
!pip install lxml
from lxml import etree, objectify
import xml.dom.minidom
!pip install wikipedia
import wikipedia
wikipedia.set_lang('sr')
import requests

In [ ]:
doc = xml.dom.minidom.parse('destination.xml')
elements = doc.getElementsByTagName('DestinationDto')
#for elem in elements:
 #   print(dir(elem))
  #  print (elem.firstChild.getElementsByTagNameNS)
   # break

In [ ]:
with open('destination.xml', encoding="utf-8") as fobj:
    xml = fobj.read()
root = etree.fromstring(xml)
print(type(root))
print(root[0])
print(root.getchildren())

<class 'lxml.etree._Element'>
<Element {http://schemas.datacontract.org/2004/07/WebApiService.Models}result at 0x1b6b795df00>
[<Element {http://schemas.datacontract.org/2004/07/WebApiService.Models}result at 0x1b6b795df40>]


In [ ]:
a = root.findall("{http://schemas.datacontract.org/2004/07/WebApiService.Models}result//{http://schemas.datacontract.org/2004/07/WebApiService.Models}DestinationDto")
destinations = {}

for elem in a:
    num = elem.getchildren()[0].text
    city = elem.getchildren()[1].text
    if num in destinations:
        continue
    else:
        destinations[num] = city



In [ ]:
destinations_cyr = {} #cyrillic list

for elem in a:
    num = elem.getchildren()[0].text
    city = elem.getchildren()[1].text
    destinations_cyr[num] = city
#destinations_cyr

In [ ]:
stations = pd.DataFrame(data=destinations.values(),index=destinations.keys(), columns=['city']).sort_values(by=['city'])
cyr_stations = pd.DataFrame(data=destinations_cyr.values(),index=destinations_cyr.keys(), columns=['cyr_name']).sort_values(by=['cyr_name'])

In [ ]:
stations = stations.join(cyr_stations).reset_index(drop=True)
stations

,city,cyr_name
0,13. OKTOBRA P,13. ОКТОБРА П
1,A.RAMPA,А.РАМПА
2,ADA AS,АДА АС
3,ADA ŽELEZNIČKA STANIC,АДА ЖЕЛЕЗНИЧКА СТАНИЦ
4,AERODROM SOMBOR,АЕРОДРОМ СОМБОР
...,...,...
2359,ŽUJOVIĆA ZABRAN,ЖУЈОВИЋА ЗАБРАН
2360,ŽUKOVAC R,ЖУКОВАЦ Р
2361,ŽUPANJAC,ЖУПАЊАЦ
2362,ŽUR,ЖУР


In [ ]:
def lowercase(item):
    return item.lower()
stations['city'] = stations['city'].apply(lowercase)
stations

,city,cyr_name
0,13. oktobra p,13. ОКТОБРА П
1,a.rampa,А.РАМПА
2,ada as,АДА АС
3,ada železnička stanic,АДА ЖЕЛЕЗНИЧКА СТАНИЦ
4,aerodrom sombor,АЕРОДРОМ СОМБОР
...,...,...
2359,žujovića zabran,ЖУЈОВИЋА ЗАБРАН
2360,žukovac r,ЖУКОВАЦ Р
2361,županjac,ЖУПАЊАЦ
2362,žur,ЖУР


Done! We've got a list of bus destinations from Belgrade

In [ ]:
countries= ['Аустрија', 'Белгија', 'Босна и Херцеговина', 'Бугарска', 'Грчка', 'Данска', 'Италија', 'Македонија', 'Мађарска', 'Немачка', 'Република Српска', 'Словачка', 'Словенија', 'Србија', 'Турска', 'Француска', 'Холандија', 'Хрватска', 'Црна Гора', 'Чешка', 'Швајцарска','Шведска']
len(countries)

22

There are 22 countries in this list, let's start from Serbia

### Now, we should find GPS coordinates

 of those cities to pop them on the map. Fortunately, there is a website: https://simplemaps.com/data/rs-cities

In [ ]:
coords = pd.read_csv('rs.csv')
# only 178 cities
cols = ['city', 'lat', 'lng']
coords = coords[cols]
coords['city'] = coords['city'].apply(lowercase) 
cities = coords['city']
cities

0         belgrade
1         novi sad
2              niš
3            zemun
4       kragujevac
          ...     
173       barajevo
174    niška banja
175      lazarevac
176          sopot
177       kostolac
Name: city, Length: 178, dtype: object

In [ ]:
# now we should write a cicle before merging
for city in cities:
    for i in range(len(stations)):
        if stations['city'][i].find(city) > -1:
            stations['city'][i] = city
            break

stations

,city,cyr_name
0,13. oktobra p,13. ОКТОБРА П
1,a.rampa,А.РАМПА
2,ada,АДА АС
3,ada železnička stanic,АДА ЖЕЛЕЗНИЧКА СТАНИЦ
4,bor,АЕРОДРОМ СОМБОР
...,...,...
2359,žujovića zabran,ЖУЈОВИЋА ЗАБРАН
2360,žukovac r,ЖУКОВАЦ Р
2361,županjac,ЖУПАЊАЦ
2362,žur,ЖУР


In [ ]:
#There is a mistake with 'bor' and 'aerodrom sombor'
stations.query('city == "sombor"')
stations = stations.loc[stations['cyr_name']!= 'АЕРОДРОМ СОМБОР']
stations

,city,cyr_name
0,13. oktobra p,13. ОКТОБРА П
1,a.rampa,А.РАМПА
2,ada,АДА АС
3,ada železnička stanic,АДА ЖЕЛЕЗНИЧКА СТАНИЦ
5,aleksandrija italia,АЛЕКСАНДРИЈА ИТАЛИА
...,...,...
2359,žujovića zabran,ЖУЈОВИЋА ЗАБРАН
2360,žukovac r,ЖУКОВАЦ Р
2361,županjac,ЖУПАЊАЦ
2362,žur,ЖУР


In [ ]:
stations_coords = stations.merge(coords, on='city', how='inner')
stations_coords = stations_coords.drop_duplicates(subset=['city'])

### Let's add wikipedia links wia wikipedia API

In [ ]:
# using requests
query = 'ada (grad)'
 
URL = 'https://sr.wikipedia.org/w/api.php' #global variable
PARAMS = {
            'action':'query',
            'format':'json',
            'list':'search',
            'utf8':1,
            'srsearch':query
        }
 
data = requests.get(URL, params=PARAMS).json()
result1 = data['query']['search'][0]
print(data['query']['search'][0])
print(result1['title'], ' - Word count: ', result1['wordcount'])

{'ns': 0, 'title': 'Ада (град)', 'pageid': 33728, 'size': 12609, 'wordcount': 622, 'snippet': 'погледајте <span class="searchmatch">Ада</span> (вишезначна одредница). <span class="searchmatch">Ада</span> је градско насеље и седиште истоимене општине, у Севернобанатском округу, у Србији, иако се <span class="searchmatch">град</span> географски', 'timestamp': '2022-02-03T09:41:58Z'}
Ада (град)  - Word count:  622


In [ ]:
#using wikipedia library
result = wikipedia.search('Ada (grad)', results=1, suggestion=False)

print(wikipedia.summary(result, sentences=2))
print(wikipedia.page(result).url)


Ада је градско насеље и седиште истоимене општине, у Севернобанатском округу, у Србији, иако се град географски налази у Бачкој, односно на десној обали реке Тисе. Претходни назив места је био Острово (на Тиси).
https://sr.wikipedia.org/wiki/%D0%90%D0%B4%D0%B0_(%D0%B3%D1%80%D0%B0%D0%B4)


In [ ]:
wiki_data = stations_coords.loc[stations_coords['city']!= 'kamenica']
print(wiki_data.sort_values(by=['city'], ignore_index=True)[40:59])

          city           cyr_name      lat      lng
40    ivanjica  ГРАБОВИЦА ИВАЊИЦА  43.5811  20.2297
41    jagodina        ЈАГОДИНА АС  43.9750  21.2564
42     kanjiža          КАЊИЖА АС  46.0667  20.0500
43     kikinda         КИКИНДА АС  45.8244  20.4592
44     kladovo  ГРАБОВИЦА КЛАДОВО  44.6039  22.6072
45        klek               КЛЕК  45.4197  20.4747
46        knić               КНИЋ  43.9167  20.7167
47   knjaževac    БОШЕВО КЊАЖЕВАЦ  43.5000  22.4333
48   koceljeva        КОЦЕЉЕВА АС  44.4708  19.8070
49    kosjerić     КАОНА КОСЈЕРИЋ  44.0000  19.9167
50    kostolac        КОСТОЛАЦ АС  44.7167  21.1667
51    kovačica         КОВАЧИЦА 1  45.1117  20.6214
52       kovin          КОВИН  АС  44.7475  20.9761
53  kragujevac      КРАГУЈЕВАЦ АС  44.0142  20.9394
54    kraljevo        КРАЉЕВО  АС  43.7234  20.6870
55     krupanj     БАЊЕВАЦ КРУПАЊ  44.3656  19.3619
56    kruševac    КП ДОМ КРУШЕВАЦ  43.5833  21.3267
57        kula            КУЛА АС  45.6109  19.5274
58  kuršumli

In [ ]:
#Let's make some functions
def create_summary(row):
    try:
        try:
            query = row['city']+' (grad)'
            result = wikipedia.search(query, results=1, suggestion=False)
            row['summary'] = wikipedia.summary(result, sentences=2)
        except:
            query = row['city']
            result = wikipedia.search(query, results=1, suggestion=False)
            row['summary'] = wikipedia.summary(result, sentences=2)
    except:
        query = row['cyr_name']
        result = wikipedia.search(query, results=1, suggestion=False)
        row['summary'] = wikipedia.summary(result, sentences=2)
    return row['summary']

wiki_data['summary'] = wiki_data.apply(create_summary, axis=1)
wiki_data.head(3)

C:\Users\dande\anaconda3\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file C:\Users\dande\anaconda3\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
C:\Users\dande\AppData\Local\Temp/ipykernel_7184/3210707345.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-

,city,cyr_name,lat,lng,summary
0,ada,АДА АС,45.8014,20.1222,Ада је градско насеље и седиште истоимене општ...
1,aleksandrovac,АЛЕКСАНДРОВАЦ АС,43.4553,21.0514,Александровац (раније Босански Александровац) ...
2,niš,АЛЕКСАНДРОВО НИШ,43.3192,21.8961,Ниш је највећи град на југу Републике Србије и...


In [ ]:
def create_url(row):
    try:
        try:
            query = row['city']+' (grad)'
            result = wikipedia.search(query, results=1, suggestion=False)
            row['url'] = wikipedia.page(result).url
        except:
            query = row['city']
            result = wikipedia.search(query, results=1, suggestion=False)
            row['url'] = wikipedia.page(result).url
    except:
        query = row['cyr_name']
        result = wikipedia.search(query, results=1, suggestion=False)
        row['url'] = wikipedia.page(result).url
    return row['url']
wiki_data['url'] = wiki_data.apply(create_url, axis=1)
wiki_data.head(3)

C:\Users\dande\anaconda3\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file C:\Users\dande\anaconda3\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
C:\Users\dande\AppData\Local\Temp/ipykernel_7184/437710099.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-c

,city,cyr_name,lat,lng,summary,url
0,ada,АДА АС,45.8014,20.1222,Ада је градско насеље и седиште истоимене општ...,https://sr.wikipedia.org/wiki/%D0%90%D0%B4%D0%...
1,aleksandrovac,АЛЕКСАНДРОВАЦ АС,43.4553,21.0514,Александровац (раније Босански Александровац) ...,https://sr.wikipedia.org/wiki/%D0%90%D0%BB%D0%...
2,niš,АЛЕКСАНДРОВО НИШ,43.3192,21.8961,Ниш је највећи град на југу Републике Србије и...,https://sr.wikipedia.org/wiki/%D0%9D%D0%B8%D1%88


In [ ]:
def create_wordcount(row):
    try:
        try:
            query = row['city']+' (grad)'
            result = wikipedia.search(query, results=1, suggestion=False)
        except:
            query = row['city']
            result = wikipedia.search(query, results=1, suggestion=False)
    except:
        query = row['cyr_name']
        result = wikipedia.search(query, results=1, suggestion=False)
    params = {
            'action':'query',
            'format':'json',
            'list':'search',
            'utf8':1,
            'srsearch':query
        }
    data = requests.get(URL, params=params).json()    
    api_result = data['query']['search'][0]
    row['wordcount'] = api_result['wordcount']
    return row['wordcount']

wiki_data['wordcount'] = wiki_data.apply(create_wordcount, axis=1)
wiki_data.head(3)

C:\Users\dande\AppData\Local\Temp/ipykernel_7184/2181079668.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wiki_data['wordcount'] = wiki_data.apply(create_wordcount, axis=1)


,city,cyr_name,lat,lng,summary,url,wordcount
0,ada,АДА АС,45.8014,20.1222,Ада је градско насеље и седиште истоимене општ...,https://sr.wikipedia.org/wiki/%D0%90%D0%B4%D0%...,622
1,aleksandrovac,АЛЕКСАНДРОВАЦ АС,43.4553,21.0514,Александровац (раније Босански Александровац) ...,https://sr.wikipedia.org/wiki/%D0%90%D0%BB%D0%...,108
2,niš,АЛЕКСАНДРОВО НИШ,43.3192,21.8961,Ниш је највећи град на југу Републике Србије и...,https://sr.wikipedia.org/wiki/%D0%9D%D0%B8%D1%88,14236


 ### Creating a map

In [ ]:
#creating a map
m = folium.Map(
    location=[44.5, 20.3],
    zoom_start=8.5)

In [ ]:
for _, city in wiki_data.iterrows():
    folium.Marker(
        location=[city['lat'], city['lng']],
        tooltip=city['city'],
        icon = folium.Icon(icon='circle'),
        popup="""
                  <i>Wordcount: </i> <br> <b>{}</b> <br> 
                  <i>url:  <a href={}>Wiki page</a>
                  <i>summary: </i><b><br>{}</b><br>""".format(
                    city['wordcount'], 
                    city['url'],
                    city['summary'],)).add_to(m)
    
m

Conclusion: so we have an interactive map of Serbia with cities accessible by bus. When you click on the marker of the city you can see the number of words on the Wikipedia page (there was an opinion: the bigger is Wikipedia page, the more interesting is the city), the Wikipedia link and a summary. Might be useful for travelling in Serbia by bus.

Plan:
1. Add markers in other countries
2. Think about bus schedule
3. Some Wikipedia links aren't correct, improve the accuracy of those links